In [ ]:
# !pip freeze | grep scikit-learn

In [ ]:
import pickle
import pandas as pd

In [ ]:
with open('model.bin', 'rb') as f_in:
    dv, lr = pickle.load(f_in)

In [ ]:
categorical = ['PUlocationID', 'DOlocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.dropOff_datetime - df.pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

In [ ]:
df = read_data('https://nyc-tlc.s3.amazonaws.com/trip+data/fhv_tripdata_2021-02.parquet')

In [ ]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_val = df.duration.values
y_pred = lr.predict(X_val)

In [16]:
y_pred.mean()

16.191691679979066

In [17]:
df['ride_id'] = f'{2021:04d}/{2:02d}_' + df.index.astype('str')

In [19]:
df['predictions'] = y_pred

In [20]:
df_result = df[['ride_id', 'predictions']]

In [21]:
df_result.to_parquet(
    "./result.parquet",
    engine='pyarrow',
    compression=None,
    index=False
)

,ride_id,predictions
1,2021/02_1,14.539865
2,2021/02_2,13.740422
3,2021/02_3,15.593339
4,2021/02_4,15.188118
5,2021/02_5,13.817206
...,...,...
1037687,2021/02_1037687,12.433246
1037688,2021/02_1037688,12.910885
1037689,2021/02_1037689,15.834923
1037690,2021/02_1037690,16.783176


```bash
!ls -lh
-rw-r--r--  1 [************]  staff    19M Jun 22 11:46 result.parquet
```